# Crime Detection Model Generator

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import joblib


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Rajeevred\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Rajeevred\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Rajeevred\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Rajeevred\AppData\Roaming\Python\Python310\site-packages\traitlets\c

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Rajeevred\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Rajeevred\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Rajeevred\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Rajeevred\AppData\Roaming\Python\Python310\site-packages\traitlets\c

AttributeError: _ARRAY_API not found

In [2]:
# Load and preprocess data
df = pd.read_csv('crime_data.csv')
df.columns = df.columns.str.strip()

df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=False)
df['hour'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.hour.fillna(0).astype(int)
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['weekday'] = df['Date'].dt.weekday

In [3]:
# Encode categorical variables
le_location = LabelEncoder()
df['Location_encoded'] = le_location.fit_transform(df['Locality_Name'])
joblib.dump(le_location, 'location_encoder.pkl')

le_crime = LabelEncoder()
df['Crime_encoded'] = le_crime.fit_transform(df['Crime_Type'])
joblib.dump(le_crime, 'crime_encoder.pkl')

['crime_encoder.pkl']

In [4]:
# Feature selection and scaling
X = df[['Location_encoded', 'Crime_encoded', 'month', 'day', 'weekday', 'hour']].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.pkl')
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [5]:
# Build Conv1D + BiLSTM Autoencoder
input_layer = tf.keras.Input(shape=(6, 1))
x = tf.keras.layers.Conv1D(32, kernel_size=2, padding='same', activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.UpSampling1D(size=2)(x)
x = tf.keras.layers.Conv1D(1, kernel_size=2, padding='same', activation='sigmoid')(x)

autoencoder = tf.keras.Model(inputs=input_layer, outputs=x)
from tensorflow.keras.losses import MeanSquaredError
autoencoder.compile(optimizer='adam', loss=MeanSquaredError())

In [6]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, verbose=1, callbacks=[early_stop])

Epoch 1/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 0.0354
Epoch 2/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 0.0014
Epoch 3/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.0011
Epoch 4/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 9.2999e-04
Epoch 5/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 7.9058e-04
Epoch 6/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - loss: 6.8307e-04
Epoch 7/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 5.8143e-04
Epoch 8/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - loss: 5.0695e-04
Epoch 9/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - loss: 4.4000e-04
Epoch 10/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - loss: 3.9648e-04
Epoch 11/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 32s 23ms/step - loss: 3.5967e-04
Epoch 12/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - loss: 3.3563e-04
Epoch 13/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - loss: 3.0891e-04
Epoch 14/30
1255/1255 ━━━━━━━━━━━

In [7]:
# Save model
autoencoder.save('conv_bilstm_autoencoder.h5')

In [8]:
# Predict and calculate reconstruction error
reconstructed = autoencoder.predict(X_scaled, verbose=0)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=(1, 2))
df['reconstruction_error'] = mse

In [9]:
# Identify anomalies
threshold = np.percentile(mse, 95)
anomalies = df[df['reconstruction_error'] > threshold]
print("\nTop anomalies based on reconstruction error:")
print(anomalies[['Locality_Name', 'Crime_Type', 'Date', 'Time', 'reconstruction_error']].head())


Top anomalies based on reconstruction error:
   Locality_Name           Crime_Type                Date              Time  \
0       Ameerpet       IDENTITY THEFT 2020-01-01 00:00:00  01-01-2020 01:11   
16    gachibowli         DRUG OFFENSE 2020-01-01 16:00:00  02-01-2020 02:57   
17    Patancheru                ARSON 2020-01-01 17:00:00  01-01-2020 23:09   
37         Uppal          SHOPLIFTING 2020-01-02 13:00:00  02-01-2020 18:04   
52    gachibowli  PUBLIC INTOXICATION 2020-01-03 04:00:00  03-01-2020 22:01   

    reconstruction_error  
0               0.000267  
16              0.000666  
17              0.000273  
37              0.000361  
52              0.000426  
